# Prerequisites

NOTE: If running this after training the model, shut down the other kernel to make sure that there is enough memory to initialize cuDNN here.

In [ ]:
# Make sure that the latest version of the package is installed (may not happen if no wheels support the version of Python installed)
!pip install tflite-support

In [1]:
# Disable tensorflow warnings to clean up cell output
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
import tensorflow as tf
import keras
import numpy as np
from tflite_support.task.vision import ImageClassifier, TensorImage

In [3]:
tf.get_logger().setLevel('ERROR')

# Convert model

In [4]:
# We load the Keras model as we will reuse it in testing later
model = keras.models.load_model("car_detection_model")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

In [5]:
# Save model
model_file = 'object_labeler.tflite'
with open(model_file, 'wb') as f:
  f.write(tflite_model)
print("Saved model to {}".format(model_file))

Saved model to object_labeler.tflite


# Add metadata

In [6]:
# This was saved earlier in mlkit_model_training.ipynb
label_map_file = "cars196_label_map.txt"

In [7]:
# Add metadata (taken from https://www.tensorflow.org/lite/models/convert/metadata)
from tflite_support import flatbuffers
from tflite_support import metadata as _metadata
from tflite_support import metadata_schema_py_generated as _metadata_fb

model_meta = _metadata_fb.ModelMetadataT()
model_meta.name = "MobileNetV3 image classifier"
model_meta.description = ("Identify the most prominent object in the "
                          "image from a set of 196 categories of cars.")
model_meta.version = "v1"
model_meta.author = "Kevin Hung"
model_meta.license = ("Apache License. Version 2.0 "
                      "http://www.apache.org/licenses/LICENSE-2.0.")

In [8]:
# Add the following metadata to input tensor TensorMetadata as MLKit input tensor is of type kTfLiteFloat32
# NormalizationOptions:
# Set normalization to [0, 255] to match MobileNetV3 with preprocessing layer
input_meta = _metadata_fb.TensorMetadataT()
input_meta.name = "image"
input_meta.description = (
    "Input image to be classified. The expected image is {0} x {1}, with "
    "three channels (red, blue, and green) per pixel. Each value in the "
    "tensor is a single float between 0 and 255.".format(224, 224))
input_meta.content = _metadata_fb.ContentT()
input_meta.content.contentProperties = _metadata_fb.ImagePropertiesT()
input_meta.content.contentProperties.colorSpace = (
    _metadata_fb.ColorSpaceType.RGB)
input_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.ImageProperties)
input_normalization = _metadata_fb.ProcessUnitT()
input_normalization.optionsType = (
    _metadata_fb.ProcessUnitOptions.NormalizationOptions)
input_normalization.options = _metadata_fb.NormalizationOptionsT()
# Don't normalize as it will be done in the preprocessing layer of the model
input_normalization.options.mean = [0.]
input_normalization.options.std = [1.]
input_meta.processUnits = [input_normalization]
input_stats = _metadata_fb.StatsT()
input_stats.max = [255]
input_stats.min = [0]
input_meta.stats = input_stats

In [9]:
# Add the following metadata to output tensor TensorMetadata
# label map as AssociatedFile with type TENSOR_AXIS_LABELS
# default score threshold as ProcessUnit with ScoreThresholdingOptions

output_meta = _metadata_fb.TensorMetadataT()
output_meta.name = "probability"
output_meta.description = "Probabilities of the labels."
output_meta.content = _metadata_fb.ContentT()
output_meta.content.contentProperties = _metadata_fb.FeaturePropertiesT()
output_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.FeatureProperties)
output_stats = _metadata_fb.StatsT()
output_stats.max = [1.0]
output_stats.min = [0.0]
output_meta.stats = output_stats
label_file = _metadata_fb.AssociatedFileT()
label_file.name = os.path.basename(label_map_file)
label_file.description = "Labels for objects that the model can recognize."
label_file.type = _metadata_fb.AssociatedFileType.TENSOR_AXIS_LABELS
output_meta.associatedFiles = [label_file]

# Threshold on 0.9 score to prevent spurious labeling
output_thresholding = _metadata_fb.ProcessUnitT()
output_thresholding.optionsType = (
    _metadata_fb.ProcessUnitOptions.ScoreThresholdingOptions)
output_thresholding.options = _metadata_fb.ScoreThresholdingOptionsT()
output_thresholding.options.globalScoreThreshold = 0.9
output_meta.processUnits = [output_thresholding]

In [10]:
# Creates subgraph info.
subgraph = _metadata_fb.SubGraphMetadataT()
subgraph.inputTensorMetadata = [input_meta]
subgraph.outputTensorMetadata = [output_meta]
model_meta.subgraphMetadata = [subgraph]

b = flatbuffers.Builder(0)
b.Finish(
    model_meta.Pack(b),
    _metadata.MetadataPopulator.METADATA_FILE_IDENTIFIER)
metadata_buf = b.Output()

# Pack
populator = _metadata.MetadataPopulator.with_model_file(model_file)
populator.load_metadata_buffer(metadata_buf)
populator.load_associated_files([label_map_file])
populator.populate()

# Confirm that the TFLite model works like the TF model

Load sample data:

In [11]:
# Load image as size of input (224, 224)
image = tf.keras.utils.load_img('example.png', target_size=(224, 224))
x = tf.keras.utils.img_to_array(image)
# Make single instance into a batch
x = np.array([x])

Predict with original TensorFlow model:

In [12]:
# Map labels
with open(label_map_file, 'r') as f:
    classes = f.read().splitlines()

In [13]:
predictions = model.predict(x)
class_idx = np.argmax(predictions[0])
print("TF prediction: {0} ({1}) @ {2:0.3f}".format(classes[class_idx], class_idx, np.max(predictions[0])))

1/1 [==============================] - 2s 2s/step
TF prediction: Cadillac SRX SUV 2012 (51) @ 0.994


Predict with TensorFlow Lite model:

In [14]:
# https://www.tensorflow.org/lite/api_docs/python/tf/lite/Interpreter
# We load from the file as it's the "final" version
interpreter = tf.lite.Interpreter(model_path=model_file)
interpreter.allocate_tensors()
output = interpreter.get_output_details()[0]
input = interpreter.get_input_details()[0]
interpreter.set_tensor(input['index'], x)
interpreter.invoke()
predictions = interpreter.get_tensor(output['index'])
print("TFLite prediction: {0} ({1}) @ {2:0.3f}".format(classes[class_idx], class_idx, np.max(predictions[0])))

TFLite prediction: Cadillac SRX SUV 2012 (51) @ 0.994


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Check that the metadata has been added correctly:

In [15]:
displayer = _metadata.MetadataDisplayer.with_model_file(model_file)
print("metadata:")
print(displayer.get_metadata_json())
print("associated files:")
print(displayer.get_packed_associated_file_list())

metadata:
{
  "name": "MobileNetV3 image classifier",
  "description": "Identify the most prominent object in the image from a set of 196 categories of cars.",
  "version": "v1",
  "subgraph_metadata": [
    {
      "input_tensor_metadata": [
        {
          "name": "image",
          "description": "Input image to be classified. The expected image is 224 x 224, with three channels (red, blue, and green) per pixel. Each value in the tensor is a single float between 0 and 255.",
          "content": {
            "content_properties_type": "ImageProperties",
            "content_properties": {
              "color_space": "RGB"
            }
          },
          "process_units": [
            {
              "options_type": "NormalizationOptions",
              "options": {
                "mean": [
                  0.0
                ],
                "std": [
                  1.0
                ]
              }
            }
          ],
          "stats": {
            "m

Check that the model works end-to-end with the TFLite API calls:

In [16]:
# ImageClassifier usage: https://www.tensorflow.org/lite/inference_with_metadata/task_library/image_classifier
tensorimg = TensorImage.create_from_file('example.png')
classifier = ImageClassifier.create_from_file(model_file)
prediction = classifier.classify(tensorimg)
if len(prediction.classifications[0].categories) > 0:
    result = prediction.classifications[0].categories[0]
else:
    result = prediction.classifications[0].categories
print(result)

Category(index=51, score=0.9874402284622192, display_name='', category_name='Cadillac SRX SUV 2012')


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
